### Gestire escrow condizionali sulla blockchain XRP Ledger.
Gli escrow condizionali permettono di bloccare fondi che possono essere rilasciati solo quando si soddisfano determinate condizioni, in questo caso definite da un fulfillment crittografico.

- condition: è una "promessa" crittografica, rappresentata come un hash crittografico, che definisce i requisiti per sbloccare l'escrow. È un valore derivato matematicamente da un dato segreto (il fulfillment);

- fulfillment: è la "chiave" crittografica necessaria per soddisfare la condizione e sbloccare l'escrow. È il segreto utilizzato per dimostrare che la condizione è stata rispettata. È il dato originale usato per generare la condizione e dev'essere fornito dal destinatario o dal completatore della transazione.

### Import

In [15]:
import xrpl
from xrpl.clients import JsonRpcClient
from xrpl.wallet import Wallet
from datetime import datetime # per calcolare le date di scadenza
from xrpl.models.transactions import EscrowCreate, EscrowFinish
from os import urandom # per generare casualmente i dati della condizione crittografica
from cryptoconditions import PreimageSha256
import asyncio

### testnet_url

In [2]:
testnet_url = "https://s.altnet.rippletest.net:51234"

### Funzioni

In [4]:
# crea un array di byte casuali (32 bit)
def generate_condition():
    randy = urandom(32)
    fulfillment = PreimageSha256(preimage=randy) # crea la condizione crittografica
    
    # restituisce la condizione in formato esadecimale che dev'essere inclusa nella transazione di escrow,
    # nonchè il fulfillment per completare l'escrow
    return (fulfillment.condition_binary.hex().upper(),
            fulfillment.serialize_binary().hex().upper())

In [16]:
# aggiunge un numero di secondi alla data corrente (si parte dal 1 gennaio 2000)
# si fa per generare il timestamp Ripple
def add_seconds(numOfSeconds):
    new_date = datetime.now()
    if new_date != '':
        new_date = xrpl.utils.datetime_to_ripple_time(new_date)
        new_date = new_date + int(numOfSeconds)
    return new_date

In [28]:
# creazione di un Escrow Condizionale
def create_conditional_escrow(seed, amount, destination, cancel, condition):
    wallet = Wallet.from_seed(seed)
    client = JsonRpcClient(testnet_url)
    cancel_date = add_seconds(cancel) # data di scadenza oltre la quale l'escrow può essere cancellato
    #finish_date = cancel_date - 200 # data di completamento impostata 200 secondi prima della scadenza
    
    # creazione della transazione EscrowCreate
    escrow_tx=xrpl.models.transactions.EscrowCreate(
        account=wallet.address,
        amount=amount,
        destination=destination,
        cancel_after=cancel_date,
        condition=condition
    )
    
    # invio della transazione: si attende che venga convalidata
    try:
        print("Invio della transazione EscrowCreate...")
        response = xrpl.transaction.submit_and_wait(escrow_tx, client, wallet)
        print("Escrow creato con successo")
        reply = response.result
    except xrpl.transaction.XRPLReliableSubmissionException as e:
        #reply = f"Submit failed: {e}"
        print(f"Errore nella creazione dell'escrow: {e}")
        return None

In [29]:
# completamento dell'Escrow
def finish_conditional_escrow(seed, owner, sequence, condition, fulfillment):
    wallet = Wallet.from_seed(seed)
    client = JsonRpcClient(testnet_url)
    finish_tx = xrpl.models.transactions.EscrowFinish(
        account=wallet.address,
        owner=owner,
        offer_sequence=int(sequence), # numero di sequenza della transazione EscrowCreate
        condition=condition,
        fulfillment=fulfillment
    )

    # sottomette la transazione e attende il risultato
    reply=""
    try:
        print("Invio della transazione EscrowFinish...")
        response = xrpl.transaction.submit_and_wait(finish_tx,client,wallet)
        print("Escrow completato con successo!")
        reply = response.result
    except xrpl.transaction.XRPLReliableSubmissionException as e:
        #reply=f"Submit failed: {e}"
        #return reply
        print(f"Errore nel completamento dell'escrow: {e}")
        return None

### MAIN

In [30]:
def main():
    # CONFIGURAZIONE INIZIALE
    print("\n--- Configurazione iniziale ---")
    
    # Inserisci le tue credenziali di test (seed del wallet sorgente)
    seed = "sn3nxiW7v8KXzPzAqzyHXbSSKNuN9"
    destination = "rPEPPER7kfTD9w2To4CQk6UCfuHM9c6GDY"
    amount = "1000"  # Importo da bloccare (in drops: 1 XRP = 1,000,000 drops)
    cancel_seconds = 3600  # Tempo di annullamento (1 ora)
    
    print(f"Seed wallet mittente: {seed}")
    print(f"Destinatario: {destination}")
    print(f"Importo: {amount} drops")
    print(f"Durata Escrow: {cancel_seconds} secondi")

    # GENERAZIONE CONDIZIONE E FULFILLMENT
    print("\n--- Generazione della condizione crittografica ---")
    condition, fulfillment = generate_condition()
    print(f"Condizione generata: {condition}")
    print(f"Fulfillment generato: {fulfillment}")

    # CREAZIONE ESCROW CONDIZIONALE
    print("\n--- Creazione dell'escrow condizionale ---")
    escrow_response = create_conditional_escrow(seed, amount, destination, cancel_seconds, condition)
    print(f"Risultato creazione escrow: {escrow_response}")

    # VERIFICA TRANSAZIONE ESCROW
    if isinstance(escrow_response, dict) and "Sequence" in escrow_response:
        sequence = escrow_response["Sequence"]
        print(f"Numero di sequenza dell'escrow: {sequence}")
    else:
        print("Errore nella creazione dell'escrow. Terminazione.")
        return

    # ATTESA SIMULATA (es. 10 secondi per test)
    import time
    print("\n--- Attesa prima di completare l'escrow (simulazione)... ---")
    time.sleep(10)

    # COMPLETAMENTO ESCROW
    print("\n--- Completamento dell'escrow condizionale ---")
    finish_response = finish_conditional_escrow(seed, wallet_from_seed(seed).address, sequence, condition, fulfillment)
    print(f"Risultato completamento escrow: {finish_response}")

    # RISULTATO FINALE
    print("\n--- Risultato finale ---")
    if isinstance(finish_response, dict) and "engine_result" in finish_response:
        print(f"Escrow completato con successo! Risultato: {finish_response['engine_result']}")
    else:
        print("Errore nel completamento dell'escrow.")

if __name__ == "__main__":
    main()


--- Configurazione iniziale ---
Seed wallet mittente: sn3nxiW7v8KXzPzAqzyHXbSSKNuN9
Destinatario: rPEPPER7kfTD9w2To4CQk6UCfuHM9c6GDY
Importo: 1000 drops
Durata Escrow: 3600 secondi

--- Generazione della condizione crittografica ---
Condizione generata: A0258020D4DEB3D56CAFB373CEF3B1D9E1DA0645BE229A30193A42BCEC39C7B744548A30810120
Fulfillment generato: A022802055EFF19F1422485DA4E827EC862B083BF692C112D910620991EB757D1968E8D7

--- Creazione dell'escrow condizionale ---
Invio della transazione EscrowCreate...


RuntimeError: asyncio.run() cannot be called from a running event loop